In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [2]:
torch.cuda.set_device(3)

In [3]:
dbunch = get_3d_dbunch_aug(Meta.df_any, bs=8)

In [4]:
x,y = dbunch.one_batch()
x.shape, y.shape

(torch.Size([8, 60, 3, 256, 256]), torch.Size([8, 60, 6]))

## Model

In [5]:
class SeqHead(nn.Module):
    def __init__(self):
        super().__init__()
        d_model = 2048*2
        self.conv = nn.Sequential(
            ConvLayer(512, 1024, stride=2),
            ConvLayer(1024, 1024, stride=2),
            ConvLayer(1024, d_model, stride=2),
            AdaptiveConcatPool2d(),
#             nn.AdaptiveAvgPool2d(1),
            Flatten()
        )
        
        self.hook = ReshapeBodyHook(self.conv)
        hidden = 96
        self.lstm = nn.LSTM(input_size=d_model*2, hidden_size=hidden, num_layers=2, bidirectional=True)
        self.head = nn.Sequential(
#             nn.Linear(hidden*2,hidden*4),
#             nn.ReLU(),
            nn.Linear(hidden*2,6)
        )
        
    def forward(self, x):
        x = self.conv(x)
        feat, _ = self.lstm(x.transpose(0,1))
        return self.head(feat.transpose(0,1))

In [6]:
m = SeqHead()

In [7]:
config=dict(custom_head=m)
learn = get_learner(dbunch, xresnet18, get_loss(), config=config)

In [8]:
hook = ReshapeBodyHook(learn.model[0])
learn.add_cb(DePadLoss())

In [9]:
learn.load(f'runs/baseline_any_xresnet18-3-full', strict=False)

In [ ]:
# learn.summary()

## Training

In [10]:
name = 'train3d_baseline_lstm'

In [11]:
# learn.lr_find()

In [12]:
do_fit(learn, 6, 1e-2, freeze=False)
learn.save(f'runs/{name}-1')

KeyboardInterrupt: 